In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers

from keras.utils import Sequence

import numpy as np

In [2]:
class pair_sequnce(
    Sequence,
    ):
    
    def __init__(
        self, 
        fl_directory, 
        skill_directory,
        pair_label_directory,
        batch_number = 10,
        ):
        
        self.batch_number = batch_number
        
        self.fl_directory = fl_directory
        self.skill_directory = skill_directory
        self.pair_label_directory = pair_label_directory
        
        
        
    def __len__(self):
        
        return int(self.batch_number)

    def __getitem__(self, batch_idx):
        
        start_idx = batch_idx*self.batch_size
        end_idx = start_idx+self.batch_size
        
        sample_idxs = range(start_idx, end_idx, 1)
        
        #print(f' {batch_idx}-th batch: {start_idx} , {end_idx}')
        
        user_idx = self.user_product_pair_idx[sample_idxs,0]
        product_idx = self.user_product_pair_idx[sample_idxs,1]
              
        batch_x = [
            self.x_user[user_idx],
            self.x_product[product_idx],            
            ]
        
        batch_y = user_product_pair_label[sample_idxs]
        
        return batch_x, batch_y
    
    def on_epoch_end(
        self,
        ):
        #print('one epoch completed')
        pass

user_number = 170
user_dim = 128

product_number = 52
product_dim = 64

class_dim = 2

user_feature = np.random.rand(
    user_number,
    user_dim
    )

product_feature = np.random.rand(
    product_number,
    product_dim
    )

user_product_pair_idx = np.array([
    [u_id, p_id]
    for p_id in range(product_number)
    for u_id in range(user_number)
    ])

user_product_pair_label = np.random.randint(
    class_dim,
    size = (len(user_product_pair_idx), )
    )
#user_product_label = to_categorical(user_product_label)

sequence = pair_sequnce(
    batch_size = 100,
    batch_number = 88,
    x_user = user_feature, 
    x_product = product_feature,
    user_product_pair_idx = user_product_pair_idx,
    user_product_pair_label = user_product_pair_label,  
    )

def build_pair_model():

    user_input = layers.Input(shape = (user_dim,))
    x_1 = layers.Dense(64, activation='relu')(user_input)

    product_input = layers.Input(shape = (product_dim,))
    x_2 = layers.Dense(64, activation='relu')(product_input)

    x_3 = layers.Dot(axes=1)([x_1, x_2])
    x_3 = layers.Activation('sigmoid')(x_3)

    model = Model(
        inputs = [user_input, product_input],
         outputs = x_3,
        )

    model.compile(
        optimizer = 'adadelta',
        loss = 'binary_crossentropy',
        metrics = [keras.metrics.AUC()],
        )

    return model

model = build_pair_model()

model.fit(
    sequence,
    batch_size = 64,
    epochs = 10,
    )